In [ ]:
# precisa ter o tesnroflow versao 2.10 no máximo pq tiraram suporte de GPU no windows por algum motivo
#!conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0
#!pip install --upgrade pip
#!pip install "tensorflow<2.11"
!pip list

In [10]:
import datetime as dt
import wget
import os
import pandas as pd
import numpy as np
from zipfile import ZipFile
from timeit import default_timer as timer
from pyts.image import GramianAngularField
from pyts.image import MarkovTransitionField
from keras.utils import load_img
from keras.utils import save_img
from keras.utils import img_to_array
from keras.utils import array_to_img
from keras.regularizers import l2
import funcoes as f
import asyncio
#os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1' # or any {'0', '1', '2'}
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing import image_dataset_from_directory
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
import plotly.express as px
from sklearn.ensemble import BaggingClassifier
from sklearn.multiclass import OneVsRestClassifier
from joblib import dump, load
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
import absl.logging
absl.logging.set_verbosity(absl.logging.ERROR)
tf.get_logger().setLevel('WARNING')

Testa se está funcionando com GPU

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


In [4]:
# define o ticker do ativo que vou usar pra treinar o modelo
ticker = "BTCUSDT"
# timeframes precisa estar em ordem crescente e começar em 1s ou 1m
timeframes = ("1m", "5m", "15m", "30m", "1h", "2h", "4h", "8h", "1d")
# quantos períodos vamos olhar pro passado
lookback = 20
# numero de quantis pra usar no markov transition field. Precisa tunar.
quantis = 3
# pega número de timeframes por minuto ou segundo
timeframes_padronizado = tuple(f.timeframes_mesma_unidade(timeframes))
# tamanho das batches de treinamento. Tem mais a ver com velocidade de processamento e não interfere muito nos resultados
batch_size = 1024#512
# set seed pros resultados não variarem
seed = 777
tf.keras.utils.set_random_seed(seed)
np.random.seed(seed)
# pega tamanho da imagem (input)
img_shape = f.image_shape()
# porcentagem dos dados que vou usar pra treino e teste
pct_imagens_teste = 0.1

# Prepara os dados

### Baixa e corrige dados

In [ ]:
# baixa os dados e junta cada timeframe e junta em 1 csv por timeframe
# retorna None se arquivo processado já existe e timeframe se ele criou um arquivo do 0
lista_processados = []
for timeframe in timeframes:
    lista_processados.append(f.baixa_e_concatena(ticker= ticker, timeframe=timeframe, ano_inicial=2017))

In [ ]:
# insere linhas sem dados e preenche com 0
# só corrige os timeframes que foram processados anteriormente
lista_p_correcao = [item for item in lista_processados if item != None]
f.corrige_arquivos(lista_p_correcao)

### Cria janelas, GAF's e salva imagens
Imagens ficam localizadas na pasta dados e são separadas por clissificação de compra ou venda

In [ ]:
# cria uma lista de numpy arrays com os dados de cada timeframe.
# Eles estão organizados na mesma ordem que os timeframes
dfs_close = [(pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", 
                      usecols=["Close time", "Close"])[::-1][["Close", "Close time"]]).to_numpy(dtype="float32") for timeframe in timeframes]
dfs_close = tuple(dfs_close)

# faz a mesma coisa pros dados de volume
dfs_volume = [(pd.read_csv(f"./Dados/Processados/BTCUSDT-{timeframe}.csv", 
                      usecols=["Close time", "Volume"])[::-1][["Volume", "Close time"]]).to_numpy(dtype="float32") for timeframe in timeframes]
dfs_volume = tuple(dfs_volume)

dfs_close[0], dfs_volume[0]

In [ ]:
if __name__ == '__main__':
    ultima_linha = len(dfs_close[0])-(lookback*timeframes_padronizado[-1])
    print("qtd de imagens para treino", ultima_linha-int(ultima_linha*pct_imagens_teste))
    print("qtd de imagens para teste", int(ultima_linha*pct_imagens_teste))
    # linhas de teste são as primeiras x%, pois os dados mais recentes vem antes
    # linhas de treino são as restantes
    linhas_teste = range(0, int(ultima_linha*pct_imagens_teste))
    linhas_treino = range(int(ultima_linha*pct_imagens_teste)+1, ultima_linha+1)
    
    # cria imagens usando multithreading
    f.roda_paralelo(linhas_treino, linhas_teste, dfs_close, dfs_volume, lookback, quantis)

# Modelos

In [15]:
# CNN com hinge loss e regularizador, se aproximando de uma SVM
def CNN_hinge(img_shape=img_shape):
    # define our MLP network
    model = keras.Sequential()
    #model.add(layers.Input(shape=(20, 180, 3)))
    model.add(layers.Input(shape=img_shape))
    
    # reescala os dados pra entre 0 e 1
    model.add(layers.Rescaling(1./255))
    
    # convolucao
    model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(1, 2)))
    model.add(layers.Dropout(0.15))
    
    # classificação
    model.add(layers.Flatten(name="flatten"))
    model.add(layers.Dense(16, activation="linear"))
    model.add(layers.Dense(8, activation="linear"))
    model.add(layers.Dense(1, kernel_regularizer=l2(0.01)))
    model.add(layers.Activation('linear'))
    
    model.compile(loss = "categorical_hinge", optimizer="Adam", metrics=["accuracy",
                                                             keras.metrics.Precision(),
                                                             keras.metrics.Recall(), 
                                                             keras.metrics.AUC()])
    return model

In [16]:
# rede CNN padrão
def CNN(img_shape=img_shape):
    model = keras.Sequential()
    model.add(layers.Input(shape=img_shape))
    
    # reescala os dados pra entre 0 e 1
    model.add(layers.Rescaling(1./255))
    
    # convolucao
    model.add(layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu'))
    model.add(layers.MaxPool2D(pool_size=(1, 2)))
    model.add(layers.Dropout(0.15))
    
    # classificação
    model.add(layers.Flatten(name="flatten"))
    model.add(layers.Dense(16, activation="linear"))
    model.add(layers.Dense(8, activation="linear"))
    model.add(layers.Dense(1, activation="sigmoid"))
    

    return model

# Treino

In [ ]:
n_modelos=5
modelos = [CNN_hinge() for i in range(n_modelos)]
resultado_CNN_hinge = np.empty((n_modelos, 5))
repeticao = 0
for model in modelos:
    # precisa definir o checkpoint antes de começar cada CNN. Se não acaba usando um do outro
    checkpoint = ModelCheckpoint("modelo", monitor='accuracy', verbose=0, save_best_only=True, mode='max')
    # para de treinar se a acuracia de treino parar de aumentar por 3 epochs
    es = EarlyStopping(monitor='accuracy', patience=4)
    callbacks_list = [checkpoint, es]

    # treina CNN hinge
    model = CNN_hinge()
    #model.compile(loss = "categorical_hinge", optimizer="Adam", metrics=["accuracy",
    #                                                                 keras.metrics.Precision(),
    #                                                                 keras.metrics.Recall(), 
    #                                                                 keras.metrics.AUC()])
    model.fit(train_data,  epochs=25, use_multiprocessing=True, verbose=0, callbacks=callbacks_list)
    model = keras.models.load_model("modelo")
    resultado_CNN_hinge[repeticao] = model.evaluate(validation_data)
    repeticao = repeticao + 1

In [ ]:
# transforma resultados CNN em df e tira a loss
resultado_CNN_hinge = pd.DataFrame(resultado_CNN_hinge, columns=["loss", "accuracy", "precision", "recall", "AUC"]).drop(columns=["loss"])
# calcula f1
resultado_CNN_hinge["f1-score"] = 2*(resultado_CNN_hinge["precision"]*resultado_CNN_hinge["recall"])/(resultado_CNN_hinge["precision"]+resultado_CNN_hinge["recall"])
# poe 0 nos nans
resultado_CNN_hinge.fillna(0, inplace=True)
# reordena as colunas
colunas = ["accuracy", "precision", "recall", "f1-score", "AUC"]
resultado_CNN_hinge = resultado_CNN_hinge[colunas]
display(resultado_CNN_hinge)

In [ ]:
# faz votacao dos resultados. depois repete pro CNN normal

In [ ]:
# Predict labels with models
labels = []
for m in models:
    predicts = np.argmax(m.predict(test), axis=1)
    labels.append(predicts)
    
# Ensemble with voting
labels = np.array(labels)
labels = np.transpose(labels, (1, 0))
labels = scipy.stats.mode(labels, axis=-1)[0]
labels = np.squeeze(labels)